## Install packages

install required python packages

In [1]:
%pip install psycopg2
%pip install pandas
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Import modules

import required python modules for data load

In [2]:
import psycopg2
import pandas as pd
import psycopg2.extras

### PostgreSQL Connector


In [3]:

class PostgresConnection(object):
    def __init__(self):
        self.connection = psycopg2.connect(database="ecomdb",
                                           user = "dhiman",
                                           password = "dhiman1234",
                                           host = "127.0.0.1",
                                           port = "5432")

    def getConnection(self):
        print("successfully connected to database")
        return self.connection

### create Star Schema in the PostgreSQL database

In [4]:
def create_tables(connection):
    commands = (
        """
        CREATE TABLE IF NOT EXISTS trans_dim (
            payment_key VARCHAR(10) PRIMARY KEY,
            trans_type VARCHAR(10) NOT NULL,
            bank_name VARCHAR(100)
        )
        """,
        """ CREATE TABLE IF NOT EXISTS item_dim (
            item_key VARCHAR(10) PRIMARY KEY,
            item_name VARCHAR(255) NOT NULL,
            item_desc VARCHAR(255) NOT NULL,
            unit_price MONEY NOT NULL,
            man_country VARCHAR(100) NOT NULL,
            supplier VARCHAR(255) NOT NULL,
            unit VARCHAR(15) NOT NULL
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS customer_dim (
            customer_key VARCHAR(10) PRIMARY KEY,
            customer_name VARCHAR(100) NOT NULL,
            contact_no VARCHAR(20) NOT NULL,
            nid VARCHAR(20) NOT NULL
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS time_dim (
            time_key VARCHAR(10) PRIMARY KEY,
            time_date VARCHAR(20) NOT NULL,
            time_hour smallint NOT NULL,
            time_day smallint NOT NULL,
            time_week VARCHAR(15) NOT NULL,
            time_month smallint NOT NULL,
            time_quarter VARCHAR(3) NOT NULL,
            time_year smallint NOT NULL
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS store_dim (
            store_key VARCHAR(10) PRIMARY KEY,
            division VARCHAR(20) NOT NULL,
            district VARCHAR(30) NOT NULL,
            upazila VARCHAR(40) NOT NULL
        )
        """,
        """
        CREATE TABLE IF NOT EXISTS fact_table (
                payment_key VARCHAR(10) NOT NULL,
                customer_key VARCHAR(10) NOT NULL,
                time_key VARCHAR(10) NOT NULL,
                item_key VARCHAR(10) NOT NULL,
                store_key VARCHAR(10) NOT NULL,
                quantity INTEGER NOT NULL,
                unit VARCHAR(15) NOT NULL,
                unit_price MONEY NOT NULL,
                total_price MONEY NOT NULL,
                PRIMARY KEY (payment_key , customer_key, time_key, item_key, store_key),
                FOREIGN KEY (payment_key)
                    REFERENCES trans_dim (payment_key)
                    ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (customer_key)
                    REFERENCES customer_dim (customer_key)
                    ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (time_key)
                    REFERENCES time_dim (time_key)
                    ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (item_key)
                    REFERENCES item_dim (item_key)
                    ON UPDATE CASCADE ON DELETE CASCADE,
                FOREIGN KEY (store_key)
                    REFERENCES store_dim (store_key)
                    ON UPDATE CASCADE ON DELETE CASCADE
        )
        """)

    cur = connection.cursor()
    # create table one by one
    for command in commands:
        cur.execute(command)
    # close communication with the PostgreSQL database server
    cur.close()
    # commit the changes
    connection.commit()


In [5]:
con = PostgresConnection().getConnection()
create_tables(connection=con)
con.close()

successfully connected to database


### Read data

read data from xlsx file using panda

In [6]:
trans_dim = pd.read_excel(r'../../data/e-commerece_data.xlsx', sheet_name='Trans_dim', engine='openpyxl')

trans_dim

,payment_key,trans_type,bank_name
0,P001,cash,NaN
1,P002,card,AB Bank Limited
2,P003,card,Bangladesh Commerce Bank Limited
3,P004,card,Bank Asia Limited
4,P005,card,BRAC Bank Limited
5,P006,card,Citizens Bank Limited
6,P007,card,City Bank Limited
7,P008,card,Community Bank Bangladesh Limited
8,P009,card,Dhaka Bank Limited
9,P010,card,Dutch-Bangla Bank Limited


In [7]:
item_dim = pd.read_excel(r'../../data/e-commerece_data.xlsx', sheet_name='Item_dim', engine='openpyxl')

item_dim

,item_key,item_name,desc,unit_price,man_country,supplier,unit
0,I00001,A&W Root Beer - 12 oz cans,a. Beverage - Soda,11.50,Netherlands,Bolsius Boxmeer,cans
1,I00002,A&W Root Beer Diet - 12 oz cans,a. Beverage - Soda,6.75,poland,CHROMADURLIN S.A.S,cans
2,I00003,Barq's Root Beer - 12 oz cans,a. Beverage - Soda,6.75,Bangladesh,DENIMACH LTD,cans
3,I00004,Cherry Coke 12oz,a. Beverage - Soda,6.75,Netherlands,Bolsius Boxmeer,cans
4,I00005,Cherry Coke Zero 12 pack,a. Beverage - Soda,6.75,Finland,HARDFORD AB,cans
...,...,...,...,...,...,...,...
259,I00260,Napkins Square Lunch,Kitchen Supplies,15.00,Cambodia,NINGBO SEDUNO IMP & EXP CO.LTD,ct
260,I00261,POM 2 ply paper towels,Kitchen Supplies,33.00,Lithuania,BIGSO AB,ct
261,I00262,Strong Everyday Napkins,Kitchen Supplies,14.00,Netherlands,Bolsius Boxmeer,ct
262,I00263,Advil 2 pill packets,Medicine,14.00,Lithuania,BIGSO AB,Ct


In [8]:
customer_dim = pd.read_excel(r'../../data/e-commerece_data.xlsx', sheet_name='Coustomer_dim', engine='openpyxl')

customer_dim

,coustomer_key,name,contact_no,nid
0,C000001,sumit,8801920345851,7505075708899
1,C000002,tammanne,8801817069329,1977731324842
2,C000003,kailash kumar,8801663795774,3769494056318
3,C000004,bhagwati prasad,8801533627961,9378834712725
4,C000005,ajay,8801943715786,3540815556323
...,...,...,...,...
9186,C009187,smt radha,8801563029753,1057824928189
9187,C009188,tulsi sharma,8801794982266,7970880905205
9188,C009189,madhubala,8801890032435,3101930298150
9189,C009190,pintu devi,8801563290767,7234328323528


In [9]:
time_dim = pd.read_excel(r'../../data/e-commerece_data.xlsx', sheet_name='Time_dim', engine='openpyxl')

time_dim

,time_key,date,hour,day,week,month,quarter,year
0,T00001,20-05-2017 14:56,14,20,3rd Week,5,Q2,2017
1,T00002,30-01-2015 22:14,22,30,4th Week,1,Q1,2015
2,T00003,14-03-2020 02:34,2,14,2nd Week,3,Q1,2020
3,T00004,27-04-2018 12:19,12,27,4th Week,4,Q2,2018
4,T00005,14-04-2018 10:43,10,14,2nd Week,4,Q2,2018
...,...,...,...,...,...,...,...,...
99994,T099995,04-01-2015 17:22,17,4,1st Week,1,Q1,2015
99995,T099996,17-01-2015 21:09,21,17,3rd Week,1,Q1,2015
99996,T099997,23-01-2020 14:42,14,23,4th Week,1,Q1,2020
99997,T099998,31-10-2015 13:52,13,31,4th Week,10,Q4,2015


In [10]:
store_dim = pd.read_excel(r'../../data/e-commerece_data.xlsx', sheet_name='Store_dim', engine='openpyxl')

store_dim

,store_key,division,district,upazila
0,S0001,SYLHET,HABIGANJ,AJMIRIGANJ
1,S0002,SYLHET,HABIGANJ,BAHUBAL
2,S0003,SYLHET,HABIGANJ,BANIACHONG
3,S0004,SYLHET,HABIGANJ,CHUNARUGHAT
4,S0005,SYLHET,HABIGANJ,HABIGANJ SADAR
...,...,...,...,...
721,S00722,BARISAL,BARISAL,HIZLA
722,S00723,BARISAL,BARISAL,BARISAL SADAR (KOTWALI)
723,S00724,BARISAL,BARISAL,MEHENDIGANJ
724,S00725,BARISAL,BARISAL,MULADI


In [11]:
fact_table = pd.read_excel(r'../../data/e-commerece_data.xlsx', sheet_name='Fact_table', engine='openpyxl')

fact_table

,payment_key,coustomer_key,time_key,item_key,store_key,quantity,unit,unit_price,total_price
0,P026,C004510,T049189,I00177,S00307,1,ct,35.0,35.0
1,P022,C008967,T041209,I00248,S00595,1,rolls,26.0,26.0
2,P030,C007261,T03633,I00195,S00496,8,ct,12.5,100.0
3,P032,C007048,T084631,I00131,S0086,8,ct,14.0,112.0
4,P014,C006430,T071276,I00050,S00488,8,cans,8.0,64.0
...,...,...,...,...,...,...,...,...,...
999995,P010,C003726,T09247,I00063,S00502,5,cans,40.0,200.0
999996,P021,C002581,T023483,I00252,S00225,1,ct,11.0,11.0
999997,P035,C006133,T039372,I00119,S00555,7,ct,53.0,371.0
999998,P031,C008735,T097975,I00257,S00440,2,ct,21.0,42.0


### Insert data

Insert data into db from pandas

#### Insert into trans_dim table

In [12]:
con = PostgresConnection().getConnection()
insert_stmt = "INSERT INTO trans_dim(payment_key, trans_type, bank_name) VALUES (%s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, trans_dim.values)
con.commit()
cur.close()
con.close()

successfully connected to database


#### Insert into customer_dim table

In [13]:
con = PostgresConnection().getConnection()
insert_stmt = "INSERT INTO customer_dim(customer_key, customer_name, contact_no, nid) VALUES (%s, %s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, customer_dim.values)
con.commit()
cur.close()
con.close()

successfully connected to database


#### Insert into item_dim table


In [14]:
con = PostgresConnection().getConnection()
insert_stmt = "INSERT INTO item_dim(item_key, item_name, item_desc, unit_price, man_country, supplier, unit) VALUES (%s, %s, %s, %s, %s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, item_dim.values)
con.commit()
cur.close()
con.close()

successfully connected to database


#### Insert into store_dim table

In [15]:
con = PostgresConnection().getConnection()
insert_stmt = "INSERT INTO store_dim(store_key, division, district, upazila) VALUES (%s, %s, %s, %s)"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, store_dim.values)
con.commit()
cur.close()
con.close()

successfully connected to database


#### Insert into time_dim table

In [17]:
con = PostgresConnection().getConnection()
insert_stmt = "INSERT INTO time_dim(time_key, time_date, time_hour, time_day, time_week, time_month, time_quarter, time_year)VALUES (%s, %s, %s, %s, %s, %s, %s, %s);"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, time_dim.values)
con.commit()
cur.close()
con.close()

successfully connected to database


#### Insert into fact_table table

In [18]:
con = PostgresConnection().getConnection()
insert_stmt = "INSERT INTO fact_table(payment_key, customer_key, time_key, item_key, store_key, quantity, unit, unit_price, total_price)VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);"
cur = con.cursor()
psycopg2.extras.execute_batch(cur, insert_stmt, fact_table.values)
con.commit()
cur.close()
con.close()

successfully connected to database
